# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [65]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [66]:
#Load the cities csv created in WeatherPy into locations dataframe and display
locations_df = pd.read_csv('../output_data/cities.csv')
locations_df


,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,lucea,18.45,-78.17,81.00,78,40,6.93,JM,1602792265
1,castro,-24.79,-50.01,64.87,93,98,6.33,BR,1602792265
2,naze,28.37,129.48,73.40,78,14,3.36,JP,1602792265
3,yar-sale,66.83,70.83,31.55,94,97,11.88,RU,1602792265
4,bitung,1.45,125.18,73.40,100,40,2.77,ID,1602792266
...,...,...,...,...,...,...,...,...,...
551,cravo norte,6.30,-70.20,90.46,56,83,3.13,CO,1602792453
552,margate,51.38,1.39,52.00,82,79,1.99,GB,1602792454
553,olafsvik,64.89,-23.71,47.43,72,98,8.88,IS,1602792454
554,ucluelet,48.93,-125.55,54.00,87,75,6.93,CA,1602792165


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [67]:
#Set the value of locations to the Lat and Lon values in locations dataframe
locations = locations_df[["Lat", "Lon"]]

#Set the heat value to the Humidity values in locations dataframe
heat = locations_df["Humidity"]

In [68]:
#Create a figure using gmaps
fig = gmaps.figure()

#Create a heat map using the locations variable above and the heat variable for the weight of the heatmap
heat_layer = gmaps.heatmap_layer(locations, weights = heat, dissipating = False, max_intensity = 10, point_radius = 1)

#Add heatmap layer to fig and display it
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [69]:
#Find indexes of locations where Max Temp is between 70 and 80, the Wind Speed is less than 10, and the Cloudiness is less than 40
ideal_index = locations_df[(locations_df["Max Temp"] < 80) & (locations_df["Max Temp"] > 70) & (locations_df["Wind Speed"] < 10) & (locations_df["Cloudiness"] < 40)].index 

#Show indexes
ideal_index

Int64Index([  2,  52,  73, 152, 197, 221, 229, 231, 234, 243, 245, 253, 257,
            264, 265, 269, 283, 298, 325, 329, 376, 396, 411, 412, 429, 437,
            472, 485, 489, 503, 515],
           dtype='int64')

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [71]:
#Create a new dataframe using the idexes of just the ideal location from above
hotel_df = locations_df.iloc[ideal_index, :]

#Generate the base url for json search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Display ideal locations pulled from above
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,naze,28.37,129.48,73.40,78,14,3.36,JP,1602792265
52,nishihara,26.18,127.76,77.00,83,20,6.93,JP,1602792280
73,vredendal,-31.67,18.50,72.32,31,0,9.60,ZA,1602792287
152,santa rosa,14.31,121.11,78.80,88,20,3.36,PH,1602792313
197,kieta,-6.22,155.63,78.98,77,36,0.60,PG,1602792328
221,sakakah,29.97,40.21,77.00,17,0,6.93,SA,1602792336
229,pyay,18.82,95.22,75.97,92,21,0.56,MM,1602792339
231,kampene,-3.60,26.67,70.75,85,4,1.32,CD,1602792339
234,grand gaube,-20.01,57.66,75.00,74,0,5.01,MU,1602792340
243,san luis,-33.30,-66.34,78.37,29,18,5.12,AR,1602792343


In [75]:
#Loop through each row in the data frame to perform json calculations
for index, row in hotel_df.iterrows():
    
    #Store lat and lon values into variable to properly pass into google places api call
    lat = row["Lat"]
    lon = row["Lon"]
    
    #Set parameters for google api call
    params = {
    "location": f"{lat}, {lon}",
    "name": "hotel",
    "radius": 5000,
    "key": g_key
}

    #Generate a call to google places api at specified location within params
    response = requests.get(base_url, params = params)
    
    #Store json data into variable
    hotel_data = response.json()
    print(f"Reading row: {index}")
    print("Gathering json data\n")


    #Try/Except statement to read hotel name at json value, then create a new column called hotel name, and store that json value in the new column. If no value found, except pushes code along
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/results....skipping.\n")

#End of print process
print("-----------------------------")
print("Data retrieval comlete!")
print("-----------------------------")

Reading row: 2
Gathering json data

Reading row: 52
Gathering json data

Reading row: 73
Gathering json data

Reading row: 152
Gathering json data

Reading row: 197
Gathering json data

Missing field/results....skipping.

Reading row: 221
Gathering json data

Reading row: 229
Gathering json data

Reading row: 231
Gathering json data

Missing field/results....skipping.

Reading row: 234
Gathering json data

Reading row: 243
Gathering json data

Reading row: 245
Gathering json data

Missing field/results....skipping.

Reading row: 253
Gathering json data

Reading row: 257
Gathering json data

Missing field/results....skipping.

Reading row: 264
Gathering json data

Reading row: 265
Gathering json data

Missing field/results....skipping.

Reading row: 269
Gathering json data

Reading row: 283
Gathering json data

Reading row: 298
Gathering json data

Reading row: 325
Gathering json data

Reading row: 329
Gathering json data

Reading row: 376
Gathering json data

Missing field/results....s

In [76]:
#Print hotel dataframe with updated hotel names
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,naze,28.37,129.48,73.40,78,14,3.36,JP,1602792265,Hotel New Amami
52,nishihara,26.18,127.76,77.00,83,20,6.93,JP,1602792280,百名伽藍
73,vredendal,-31.67,18.50,72.32,31,0,9.60,ZA,1602792287,Voorsorg Guest House
152,santa rosa,14.31,121.11,78.80,88,20,3.36,PH,1602792313,El Cielito Hotel - Sta. Rosa
197,kieta,-6.22,155.63,78.98,77,36,0.60,PG,1602792328,NaN
221,sakakah,29.97,40.21,77.00,17,0,6.93,SA,1602792336,Raoum Inn Hotel
229,pyay,18.82,95.22,75.97,92,21,0.56,MM,1602792339,Lucky Dragon Hotel
231,kampene,-3.60,26.67,70.75,85,4,1.32,CD,1602792339,NaN
234,grand gaube,-20.01,57.66,75.00,74,0,5.01,MU,1602792340,LUX Grand Gaube Resort & Villas
243,san luis,-33.30,-66.34,78.37,29,18,5.12,AR,1602792343,Hotel Premium Tower Suites San Luis


In [77]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [78]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[["Lat", "Lon"]]
markers = gmaps.marker_layer(marker_locations, hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))